# Fitting XRF maps

This notebook will create and displays fits of mapping scans.
The scans can either be single scans, or a set of scans taken across a sample to reduce the sample files.
All the scans should be from the same sample and same region, with a small overlap in motor positions.
(Basically a set of scans created by the mapping tool for a single region)

*Remember to create the calibration and configuration files using pymca!
This script will not run without them!!*

#### Note that the scans *must* be given in list form, even if it is a single scan!

In [ ]:
#Import necessary libaries

%matplotlib widget
from xrf_pyMCA_fit import xrfBatch
import numpy as np
import DanMAX as DM
import matplotlib.pyplot as plt
import h5py

In [ ]:
#Select scans to fit, must be a list!!!
scan_list = [5,6,7]

#Define parameters to find the data
proposal,visit = DM.getCurrentProposal() #Default will be collected from the current path, you can set others, if you wish to use data from a previous beamtime

#Sessionpath will tell the fitter which proposal and visit it should read data from
#This will also be used to find where to store fitted data
sessionpath = f'/data/visitors/danmax/{proposal}/{visit}/' #Default will be f'/data/visitors/danmax/{proposal}/{visit}/'

#Calibration and configuration files must be made in pyfai prior to fitting:
calib_file  = f'{sessionpath}process/pymca_calib.calib'
config_file = f'{sessionpath}process/pymca_config.cfg' 




#The following code sets up the fitter. 
#You can change if you want different things, but it should be good enough!
channel     = 0 #Default is 0
detector    = 'falconx' #Default is falconx


#Define which files output to save
make_elements_h5     = True
make_elements_tif    = False
make_pymca_h5        = True
make_spectrum_fit_h5 = True
return_fit_filename  = False

#Define the XRF batch fitter
fits= xrfBatch(session_path= sessionpath,
              scan_list=scan_list,
              config_file=config_file,
              calib_file=calib_file,
              channel=channel,
              detector=detector,
              proposal=proposal,
              visit=visit)

In [ ]:
#Load the the data and stitch them if there are multiple scans
fits.readData()

In [ ]:
#Fit the data and the average spectrum
fits.writePymcafile()
fits.fitElementsToFile(make_elements_h5,make_elements_tif,return_fit_filename)
fits.fitAvgSpectrumToFile()

## Showing the fitted maps

The following code snippits takes the fitted maps from above and plots them as images.

In [ ]:
#Define the information needed to plot the fits

#Define the h5 file containing the fits. Default is from the file just fitted above.
#If another file is neededset it here
fits_filename = f'{fits.out_dir_elements}{fits.elem_file_name}.h5'

#Define which maps to plot, the lists give the lower and upper limit.
#Set limits to "None" for default value
maps_to_show = {'Ca_K':[None, None],
               'Sr_K': [50, 130],
               'Br_K': [2, 14],
               'Zn_K': [2, 14],
               'Mn_K': [None, None]}

# Set the number of columns for the figure
cols = 3 
rows = int(len(maps_to_show)/cols) + (len(maps_to_show)%cols!=0)

#Set the base path in the h5 fit file to the fit maps
h5_fit_path = 'xrf_fits/xrf_fit/results/parameters/'

In [ ]:
# initialize subplots with shared x- and y-axes
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.tight_layout()
axs = axs.flatten() # flatten the axes list to make it easier to index

#Open file containing fitted maps
with h5py.File(fits_filename,'r') as fit_file: 
    for ii,key in enumerate(maps_to_show.keys()):
        #Load element specific data
        element = fit_file[f'{h5_fit_path}{key}']
        ax = axs[ii]
        ax.set_title(key)
        ax.set_axis_off()
        ax.set_aspect('equal')
        
        #Plot the data with pcolmesh
        if 'fits' in locals()_
            ax.pcolormesh(fits.x.reshape(element.shape),
                          fits.y.reshape(element.shape),
                          element,
                          vmin=maps_to_show[key][0],
                          vmax=maps_to_show[key][1],
                          shading='nearest')
            
        else:
            #Use imshow if the "fits" variable has been unloaded.
            ax.imshow(element,
                     vmin=maps_to_show[key][0],
                     vmax=maps_to_show[key][1])